# Water Sampling Processing Script
### Author: Andrew Reed, CGSN - WHOI

### Motivation
The motivation for this script is to automate producing the bottle files in a consistent, easily parseable manner consistent with SeaBird's naming and processing scheme. I chose to take this approach after attempting to parse the bottle (.btl) files output from SeaBird's SeaSoft V2 processing software. Unfortunately, SeaBird outputs into a tab-deliminated text format with inconsistent spacing between columns and spacing offsets. This prevents simple alignment of column names:column values and makes parsing column names more difficult without a priori knowing where parsing issues will arise. 

### Approach
I chose to utilize the rosette (.ros) files produced by SeaBird's SeaSoft V2 software as part of the initial conversion of their propietary .cnv formatted data produced by their CTDs and rosettes. The .ros files have explicit column naming outlined in the header of the file, and the columns with the parameter values are consistently spaced. This allows for an easy mapping of the column name to column values based on location. 

Additionally, I use a secondary file which outlines the parameter "short names" to the parameter "full names + units." This information is from SeaBird's SeaSoft V2 manual. Additionally, the processing method of taking the mean of all scans per bottle firing also follows the procedure outlined in the SeaSoft manual. 

### Usage
To use this software for your own processing, the following pacakges need to be installed:
* Pandas
* Numpy

Additionally, you will need to change the filepaths to the appropriate directory locations on your local machine. Once that is complete, simply run the cells in order. The software will write the results to the directroy where the rosette files are stored. 

**Note**: The ".btl" files produced here are meant for ease-of-use in producing our water sample summary sheets. They will not work for continued processing to get derived variables using SeaBird's software. 

In [1]:
# Import packages used in this notebook
import os, sys
import pandas as pd
import numpy as np

In [2]:
basepath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/'

In [3]:
# Load the name mapping for the column names
sbe_name_map = pd.read_excel('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')
sbe_name_map

,Short Name,Full Name,Friendly Name,Units,Notes/Comments
0,accM,Acceleration [m/s^2],acc M,m/s^2,NaN
1,accF,Acceleration [ft/s^2],acc F,ft/s^2,NaN
2,altM,Altimeter [m],alt M,m,NaN
3,altF,Altimeter [ft],alt F,ft,NaN
4,avgsvCM,"Average Sound Velocity [Chen-Millero, m/s]",avgsv-C M,"Chen-Millero, m/s",NaN
5,avgsvCF,"Average Sound Velocity [Chen-Millero, ft/s]",avgsv-C F,"Chen-Millero, ft/s",NaN
6,avgsvDM,"Average Sound Velocity [Delgrosso, m/s]",avgsv-D M,"Delgrosso, m/s",NaN
7,avgsvDF,"Average Sound Velocity [Delgrosso, ft/s]",avgsv-D F,"Delgrosso, ft/s",NaN
8,avgsvWM,"Average Sound Velocity [Wilson, m/s]",avgsv-W M,"Wilson, m/s",NaN
9,avgsvWF,"Average Sound Velocity [Wilson, ft/s]",avgsv-W F,"Wilson, ft/s",NaN


In [4]:
# Load the cruise i.d.
with open('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/Data/CRUISE_ID') as file:
    cruise_id = file.read().strip()
cruise_id

'ar30-03'

In [5]:
def parse_header(header):
    """
    Function to parse the header of a SeaBird rosette file (.ros).
    This takes the place of generating a bottle file (.btl) using
    SeaBird's SeaSoft software.
    
    Args:
        header - a text file containing the relevant header info
    Returns:
        header_dict - a dictionary containing a mapping of the
            column name to its position in the data file
        start_time - the time that the SBE system started recording
        scan_interval - the # of scans per second
    """
    # Initialize the header dictionary
    header_dict = {}
    start_time = []
    scan_interval = []
    for line in header.splitlines():
        if 'name' in line:
            header_index = line.split()[2]
            header_name = line.split()[4].replace(':','')
            header_dict.update({header_name:header_index})
        elif 'interval' in line:
            scan_interval = line.split()[-1]
        elif 'start_time' in line:
            start_time = line.split()[3:7] 
            
    # Return the relevant important data
    return header_dict, start_time, scan_interval

In [6]:
def parse_data(data,header_dict):
    """
    Parses the data from the rosette file based on the position of the
    column, using the column locations from the header.
    
    Args:
        data - a text file containing the data from the rosette file
        header_dict - a dictionary containing a mapping of column names to the
            column position
    Returns:
        data_dict - a dictionary containing key:value pairs of where key is the
            column position and value is the column values 
    """
    
    # Generate a dictionary for the data with mapping from the column dictionary
    data_dict = {x:[] for x in header_dict.values()}
    
    # Now parse the data
    for line in data.splitlines():
        for i,x in enumerate(line.split()):
            try:
                float(x)
                data_dict[str(i)].append(x)
            except:
                pass
    
    return data_dict

In [52]:
def generate_btl_data(data_dict, header_dict, start_time, scan_interval):
    """
    Function to generate the equivalent bottle file (.btl).
    
    Args:
        data_dict - a dictionary containing key:value pairs of where key is the
            column position and value is the column values
        header_dict - a dictionary containing a mapping of column names to the
            column position
        start_time - the time that the CTD cast started
        scan_interval - the number of seconds per ctd scan
    Returns:
        df - a pandas dataframe containing the data from the data dictionary 
            with the column names from the header dictionary and the datetime
            calculated from the start_time and the scan_interval
    """
    
    # Using the data and header dictionaries, map the data columns to the
    # appropriate column names
    result = {}
    for key,item in header_dict.items():
        values = data_dict.get(item)
        result.update({key:values})

    # Put the data into a dataframe and convert the data from strings to floats
    df = pd.DataFrame.from_dict(result)
    for column in df.columns.values:
        try:
            df[column] = df[column].apply(lambda x: float(x))
        except:
            pass
    
    # Groupby the dataframe based on bottle name
    df = df.groupby(by='nbf').mean()

    # Convert the scan counts to seconds
    df['scan'] = df['scan'].apply(lambda x: x*float(scan_interval))

    # Add in the date time
    start_time = pd.to_datetime(' '.join(start_time))
    df['Datetime'] = df['scan'].apply(lambda x: start_time + pd.to_timedelta(x,unit='s'))
    df['Datetime'] = df['Datetime'].apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%S.%fZ'))

    return df


In [53]:
def parse_cast_number(filename):
    """
    Parses the cast number out of the file name. It assumes that the 
    cast number is 3 numbers long and occurs right before the file
    extension.
    
    Args:
        filename - the name of the file to be parsed
    Returns:
        cast_num - the cast number of the file
    """
    
    index = file.index('.')
    # From the index, count backwards until have 3 numbers for cast
    num = 0
    ind = 0
    while num < 3:
        ind = ind+1
        try:
            float(file[index-ind])
            num = num+1
        except:
            pass
    # Nower return the cast number
    cast_num = file[index-ind:index]
    
    return cast_num

In [54]:
def process_ros_files(filepath,sbe_name_map,cruise_id):
    """
    Parent function to parse and process SeaBird rosette (.ros)
    files, generate a pandas dataframe, and write bottle (.btl)
    files (as csvs). This takes the place of the bottle processing
    in SeaSoft V2 provided by SeaBird.
    
    Args:
        filepath - directory path to the location of the rosette files
        sbe_name_map - a pandas dataframe containing a mapping of the
            seabird short names to full names. Taken directly from the
            seabird manuals.
        cruise_id - string input of the cruise id
    Calls:
        parse_header
        parse_data
        generate_btl_data
    Returns:
        .btl - writes a bottle file to the same directory location as the
            rosette file.
    """
    
    # First, open the file and read it in
    with open(filepath) as file:
        data = file.read()
        header, data = data.split('*END*')
        
    # Parse the header file
    header_dict, start_time, scan_interval = parse_header(header)
    
    # Parse the data based on the output from the header
    data_dict = parse_data(data, header_dict)
    
    # Create a pandas dataframe
    df = generate_btl_data(data_dict, header_dict, start_time, scan_interval)
    
    # Rename the column title using the sbe_name_mapping 
    for colname in list(df.columns.values):
        try:
            fullname = list(sbe_name_map[sbe_name_map['Short Name'] == colname]['Full Name'])[0]
            df.rename({colname:fullname},axis='columns',inplace=True)
        except:
            pass
    # Rename the index as well
    df.index.rename(list(sbe_name_map[sbe_name_map['Short Name'] == df.index.name]['Full Name'])[0],inplace=True)
    
    # Add in the cruise id
    df['Cruise ID'] = cruise_id
    
    # Parse and add in the cast number
    cast = parse_cast_number(filepath)
    df['Cast'] = cast
    
    # Generate the btl name file
    btl_path = filepath.replace('.ros','.csv')
    
    # Save to the same directory as the rosette files
    df.to_csv(btl_path)

In [55]:
# Iterating through this process will generate .btl files from the rosette files.
# This is done in lieu of using the SeaBird software processing. I chose this route
# because of inconsistent column spacings made parsing and matching column names to
# column values difficult. The rosette headers make the column relationship explicit.
filepath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/Data/CTD/'
for file in os.listdir(filepath):
    if '.ros' in file:
        process_ros_files(filepath+file, sbe_name_map, cruise_id)
        
        

In [56]:
btl = pd.read_csv('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/Data/CTD/ar30-03003.csv')

In [57]:
btl

,Bottles Fired,Modulo Error Count,"Pressure, Digiquartz [db]","Depth [salt water, m]",Latitude [deg],Longitude [deg],"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]",Conductivity [S/m],"Conductivity, 2 [S/m]",...,"Salinity, Practical, 2 [PSU]","Oxygen, SBE 43 [ml/l]","Oxygen Saturation, Garcia & Gordon [ml/l]","Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]",Scan Count,Flag,Datetime,Cruise ID,Cast
0,1.0,0.0,2998.401887,2948.939093,60.889800,-35.355969,1.351952,1.353832,3.137537,3.137507,...,34.881792,6.562714,7.742422,24.113882,0.241305,3692.419621,0.0,2018-06-07T09:20:35.419620Z,ar30-03,3
1,2.0,0.0,2540.190990,2500.937278,60.889620,-35.354850,3.005421,3.006737,3.268464,3.268433,...,34.924311,6.029555,7.426458,55.262000,-0.252421,4511.920276,0.0,2018-06-07T09:34:14.920276Z,ar30-03,3
2,3.0,0.0,2029.040691,2000.069546,60.889470,-35.353845,3.460248,3.461490,3.289796,3.289805,...,34.938716,5.951516,7.343359,55.262000,-0.207462,5581.587799,0.0,2018-06-07T09:52:04.587798Z,ar30-03,3
3,4.0,0.0,1623.636144,1601.979918,60.889330,-35.352960,3.514286,3.515012,3.274421,3.274428,...,34.895913,6.223611,7.335831,55.262000,-0.046368,6364.338425,0.0,2018-06-07T10:05:07.338424Z,ar30-03,3
4,5.0,0.0,1317.683866,1301.049608,60.889200,-35.352110,3.374191,3.374618,3.246366,3.246374,...,34.862626,6.549161,7.362676,55.262000,-0.093530,7026.713955,0.0,2018-06-07T10:16:09.713954Z,ar30-03,3
5,6.0,0.0,1014.467856,1002.383691,60.889020,-35.350980,3.484716,3.485085,3.244405,3.244420,...,34.876723,6.550929,7.342109,55.262000,-0.536274,7689.631152,0.0,2018-06-07T10:27:12.631151Z,ar30-03,3
6,7.0,0.0,812.846619,803.550990,60.888824,-35.349666,3.553446,3.553530,3.242305,3.242309,...,34.883456,6.554013,7.329478,55.262000,-0.809901,8389.923379,0.0,2018-06-07T10:38:52.923378Z,ar30-03,3
7,8.0,0.0,609.473485,602.797412,60.888730,-35.349122,3.645881,3.646366,3.242139,3.242190,...,34.890345,6.549424,7.312651,55.262000,-0.859861,8946.215490,0.0,2018-06-07T10:48:09.215490Z,ar30-03,3
8,9.0,0.0,407.482876,403.215124,60.888640,-35.348460,3.739780,3.740225,3.241322,3.241387,...,34.888597,6.548033,7.296065,55.262000,-0.992252,9504.132603,0.0,2018-06-07T10:57:27.132603Z,ar30-03,3
9,10.0,0.0,205.654742,203.599969,60.888500,-35.347760,4.048180,4.048482,3.259844,3.259899,...,34.889823,6.461930,7.241675,55.262000,-0.955559,10080.716398,0.0,2018-06-07T11:07:03.716397Z,ar30-03,3


### Create A Summary Sheet
Now, I want to generate a summary sheet from all of the processed bottle files. This includes loding all of the files into dataframes and appending. The one challenge is if some casts included columns that are unique.

In [58]:
dirpath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/Data/CTD/'
btl = pd.DataFrame()
for file in os.listdir(dirpath):
    if '.csv' in file:
        df = pd.read_csv(dirpath+file)
        btl = btl.append(df)
        

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [39]:
os.listdir(dirpath)

['ar30-03.psa',
 'ar30-03.xmlcon',
 'ar30-03001.bl',
 'ar30-03001.btl',
 'ar30-03001.csv',
 'ar30-03001.hdr',
 'ar30-03001.hex',
 'ar30-03001.ros',
 'AR30-03001.XMLCON',
 'ar30-03002.bl',
 'ar30-03002.hdr',
 'ar30-03002.hex',
 'AR30-03002.XMLCON',
 'ar30-03002b.bl',
 'ar30-03002b.btl',
 'ar30-03002b.csv',
 'ar30-03002b.hdr',
 'ar30-03002b.hex',
 'ar30-03002b.ros',
 'AR30-03002B.XMLCON',
 'ar30-03003.bl',
 'ar30-03003.btl',
 'ar30-03003.csv',
 'ar30-03003.hdr',
 'ar30-03003.hex',
 'ar30-03003.ros',
 'AR30-03003.XMLCON',
 'ar30-03004.bl',
 'ar30-03004.btl',
 'ar30-03004.csv',
 'ar30-03004.hdr',
 'ar30-03004.hex',
 'ar30-03004.ros',
 'AR30-03004.XMLCON',
 'ar30-03005.bl',
 'ar30-03005.btl',
 'ar30-03005.csv',
 'ar30-03005.hdr',
 'ar30-03005.hex',
 'ar30-03005.ros',
 'AR30-03005.XMLCON',
 'ar30-03006.bl',
 'ar30-03006.btl',
 'ar30-03006.csv',
 'ar30-03006.hdr',
 'ar30-03006.ros',
 'AR30-03006.XMLCON',
 'ar30-03007.bl',
 'ar30-03007.btl',
 'ar30-03007.csv',
 'ar30-03007.hdr',
 'ar30-03007.h

In [59]:
btl

,"Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]",Bottles Fired,Cast,Conductivity [S/m],"Conductivity, 2 [S/m]",Cruise ID,Datetime,"Depth [salt water, m]",Flag,...,"Oxygen raw, SBE 43 [V]","Oxygen, SBE 43 [ml/l]","Pressure, Digiquartz [db]",SPAR/Surface Irradiance,"Salinity, Practical [PSU]","Salinity, Practical, 2 [PSU]",Scan Count,"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]","Turbidity, WET Labs ECO [NTU]"
0,19.036789,0.857820,1.0,1,3.455169,3.454611,ar30-03,2018-06-06T09:26:41.711723Z,146.727433,0.0,...,NaN,6.285522,148.209113,NaN,34.951198,34.951134,4237.711724,6.176369,6.170356,NaN
1,19.161090,0.831087,2.0,1,3.476623,3.476591,ar30-03,2018-06-06T09:31:25.295283Z,104.726093,0.0,...,NaN,6.398124,105.772670,NaN,34.955305,34.955442,4521.295284,6.427731,6.427268,NaN
2,21.422320,0.476894,3.0,1,3.505544,3.505613,ar30-03,2018-06-06T09:36:04.753840Z,40.167485,0.0,...,NaN,6.660419,40.562536,NaN,34.964652,34.964534,4800.753841,6.765898,6.766763,NaN
3,26.667423,0.128244,4.0,1,3.540467,3.542229,ar30-03,2018-06-06T09:38:13.795610Z,23.168351,0.0,...,NaN,7.070651,23.395247,NaN,34.973422,34.968558,4929.795610,7.144649,7.168527,NaN
4,28.169660,0.137615,5.0,1,3.558928,3.558911,ar30-03,2018-06-06T09:39:31.462339Z,15.342588,0.0,...,NaN,7.059086,15.492526,NaN,34.966438,34.966319,5007.462339,7.355192,7.355116,NaN
5,22.205639,0.395238,6.0,1,3.559666,3.559626,ar30-03,2018-06-06T09:41:33.920770Z,2.713072,0.0,...,NaN,7.072604,2.739423,NaN,34.966416,34.965722,5129.920771,7.369519,7.369777,NaN
6,22.138271,0.396349,7.0,1,3.559819,3.559834,ar30-03,2018-06-06T09:41:41.962443Z,2.942814,0.0,...,NaN,7.073497,2.971546,NaN,34.966478,34.966449,5137.962444,7.371016,7.371184,NaN
0,16.659648,1.555459,1.0,002b,3.491486,3.491553,ar30-03,2018-06-06T14:24:52.333918Z,153.806866,0.0,...,NaN,6.273453,155.359567,NaN,34.976795,34.976764,731.333918,6.544079,6.544829,NaN
1,17.789664,1.171753,2.0,002b,3.496320,3.496352,ar30-03,2018-06-06T14:27:56.875732Z,98.941814,0.0,...,NaN,6.412878,99.927062,NaN,34.968881,34.969022,915.875733,6.631936,6.632142,NaN
2,17.367361,1.301403,3.0,002b,3.514970,3.515027,ar30-03,2018-06-06T14:31:07.000884Z,39.068371,0.0,...,NaN,6.566302,39.451670,NaN,34.973564,34.973786,1106.000885,6.860097,6.860499,NaN


In [51]:
btl.dropna(subset=['Turbidity, WET Labs ECO [NTU]'])

,"Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]",Bottles Fired,Cast,Conductivity [S/m],"Conductivity, 2 [S/m]",Cruise ID,Datetime,"Depth [salt water, m]",Flag,...,"Oxygen raw, SBE 43 [V]","Oxygen, SBE 43 [ml/l]","Pressure, Digiquartz [db]",SPAR/Surface Irradiance,"Salinity, Practical [PSU]","Salinity, Practical, 2 [PSU]",Scan Count,"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]","Turbidity, WET Labs ECO [NTU]"
0,NaN,NaN,1.0,6,3.139994,3.139922,ar30-03,2018-06-09T14:28:43.960736Z,NaN,0.0,...,1.797100,NaN,2688.645551,1025.728571,34.886451,NaN,3003.960736,1.513251,1.513116,0.406800
1,NaN,NaN,2.0,6,3.155050,3.155097,ar30-03,2018-06-09T14:34:25.627676Z,NaN,0.0,...,1.819927,NaN,2541.079000,903.841837,34.890100,NaN,3345.627676,1.747959,1.749167,0.362900
2,NaN,NaN,3.0,6,3.255287,3.255307,ar30-03,2018-06-09T14:59:26.170543Z,NaN,0.0,...,1.875000,NaN,2130.010714,1021.879592,34.908600,NaN,4846.170544,3.055310,3.055708,0.360500
3,NaN,NaN,4.0,6,3.285758,3.285768,ar30-03,2018-06-09T15:06:02.337527Z,NaN,0.0,...,1.922427,NaN,1822.976939,1120.361224,34.922733,NaN,5242.337527,3.524971,3.525088,0.356400
4,NaN,NaN,5.0,6,3.266032,3.266057,ar30-03,2018-06-09T15:13:41.671228Z,NaN,0.0,...,2.057873,NaN,1526.787837,825.005510,34.880163,NaN,5701.671228,3.480692,3.480645,0.358500
5,NaN,NaN,6.0,6,3.260472,3.260503,ar30-03,2018-06-09T15:21:16.338258Z,NaN,0.0,...,2.147549,NaN,1217.468286,903.384286,34.883831,NaN,6156.338258,3.561078,3.560833,0.361900
6,NaN,NaN,7.0,6,3.261812,3.261873,ar30-03,2018-06-09T15:27:03.796869Z,NaN,0.0,...,2.197500,NaN,1010.361102,925.534490,34.893990,NaN,6503.796870,3.666202,3.666147,0.365596
7,NaN,NaN,8.0,6,3.259531,3.259582,ar30-03,2018-06-09T15:32:10.672115Z,NaN,0.0,...,2.253314,NaN,811.633327,859.906531,34.895982,NaN,6810.672115,3.735627,3.735445,0.370053
8,NaN,NaN,9.0,6,3.278960,3.278960,ar30-03,2018-06-09T15:37:44.422382Z,NaN,0.0,...,2.283851,NaN,606.767755,883.848367,34.905033,NaN,7144.422382,4.045600,4.044631,0.376100
9,NaN,NaN,10.0,6,3.316194,3.316193,ar30-03,2018-06-09T15:43:28.505990Z,NaN,0.0,...,2.335790,NaN,405.744367,827.895714,34.913984,NaN,7488.505991,4.552031,4.550992,0.389100
